In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics.pairwise import paired_distances
from sklearn.preprocessing import StandardScaler
import csv
from sklearn.model_selection import GridSearchCV
from sklearn.metrics._scorer import make_scorer
from sklearn.svm import SVR
from sklearn.multioutput import MultiOutputRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import ParameterGrid
from scipy.stats import zscore

In [2]:
def custom_loss_func(ground_truth, predictions):
     return np.sum(paired_distances(ground_truth, predictions))/len(ground_truth)

In [3]:
from sklearn.feature_selection import SelectKBest, f_classif
df = pd.read_csv('development.csv', sep=',')

In [4]:
noise_pmax = []
noise_negpmax = []
noise_area = []#4.7
noise_tmax = []
noise_rms = []
noise_vpp = []
noise_slewrate = []
pmax = [f'pmax[{i}]' for i in range(18)]
negpmax = [f'negpmax[{i}]' for i in range(18)]
for i in range(0,18):
    df[f"vpp[{i}]"] = (df[f"pmax[{i}]"] - df[f"negpmax[{i}]"])
df = df[df['negpmax[1]'] < 0]
df = df[df['negpmax[2]'] < 0]
df = df[df['negpmax[3]'] < 0]
df = df[df['negpmax[4]'] < 0]
df = df[df['negpmax[5]'] < 0]
df = df[df['negpmax[6]'] < 0]
df = df[df['negpmax[8]'] < 0]
df = df[df['negpmax[9]'] < 0]
df = df[df['negpmax[10]'] < 0]
df = df[df['negpmax[11]'] < 0]
df = df[df['negpmax[13]'] < 0]
df = df[df['negpmax[14]'] < 0]
df = df[df['negpmax[15]'] < 0]
for i in [0, 7, 12, 16, 17]: 
    noise_pmax.append(f'pmax[{i}]')
    noise_area.append(f'area[{i}]')
    noise_area.append(f'negpmax[{i}]')
    noise_vpp.append(f'vpp[{i}]')
for i in range(0, 18): 
    noise_rms.append(f'rms[{i}]')
    noise_tmax.append(f'tmax[{i}]')
noise = noise_pmax + noise_negpmax + noise_area + noise_tmax + noise_rms+ noise_vpp + noise_slewrate
df.drop(columns=noise, inplace=True)
y = df[['x','y']].values
df.drop(columns=['x','y'], inplace=True)

In [5]:
df_eval = pd.read_csv('evaluation.csv', sep=',')
for i in range(0,18):
    df_eval[f"vpp[{i}]"] = (df_eval[f"pmax[{i}]"] - df_eval[f"negpmax[{i}]"])
df_eval.drop(columns=noise, inplace=True)
df = df.sample(frac=1).reset_index(drop=True)
X_train_valid = df.values
y_train_valid = y
X_test = df_eval.drop(columns='Id').values

#X_train_valid, X_valid, y_train_valid, y_valid = train_test_split(df, y, test_size=0.2, random_state=42, shuffle=True)

In [6]:
custom_scorer = make_scorer(custom_loss_func, greater_is_better=False)
param_grid = {'n_estimators': [50, 100, 200],
              'criterion': ['squared_error'],
              'max_features': ['sqrt', 'log2'],
              'random_state': [42],
              'n_jobs': [-1]      
}
gs = GridSearchCV(RandomForestRegressor(), param_grid, scoring=custom_scorer, n_jobs=-1, cv=5)
gs.fit(X_train_valid, y_train_valid)
print(gs.best_score_)  
print(gs.best_params_)  # {'criterion': 'squared_error', 'max_features': 'sqrt', 'n_estimators': 50, 'n_jobs': -1, 'random_state': 42}
print(gs.best_estimator_)
y_pred = gs.predict(X_test)
#print(f'Metric value: {custom_loss_func(y_test, y_pred)}') # Metric value: 4.742701679372149

-174.69551799415845
{'criterion': 'squared_error', 'max_features': 'sqrt', 'n_estimators': 200, 'n_jobs': -1, 'random_state': 42}
RandomForestRegressor(max_features='sqrt', n_estimators=200, n_jobs=-1,
                      random_state=42)
